In [2]:
# data comes from http://web.stanford.edu/~ericyi/project_page/part_annotation/index.html
from pyntcloud import PyntCloud
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import sys
import os
if sys.platform == 'darwin':
    data_path = os.getcwd() + "/PartAnnotation"
else:
    data_path = os.getcwd() + "\\PartAnnotation"

In [3]:
def find_data(data_path, max_file_num=None, folder_filter=(None, None)):
    """
    Find file in each folder according to the 'data_path'.
    
    Giving the max number of files via `max_file_num`, it will read first `max_file_num` in each folder. Read all if there is no enough file inside.
    
    `folder_filter` is a tuple like (100, 2000) which indicates the number of files will between 100 and 2000.
    """
    
    min_th, max_th = folder_filter
    if max_file_num is not None and folder_filter is not None:
        if min_th is not None:
            assert(max_file_num > min_th), "`max_file_num` should be greater than `" + min_th + "` in " + folder_filter
        
    data = []
    label = []
    for entry in os.scandir(data_path):
        if entry.is_dir():
            target_dir_path = os.path.join(data_path, entry.name, 'points')
            path, dirs, files = os.walk(target_dir_path).__next__()
            file_count = len(files)
            if (min_th is None or file_count >= min_th) and (max_th is None or file_count <= max_th):
                count = 0
                for pts_data in os.scandir(target_dir_path):
                    if (max_file_num is None) or (count < max_file_num):
                        data.append(os.path.join(data_path, entry.name, 'points', pts_data.name))
                        label.append(entry.name)
                        count += 1
                    else:
                        break
    return data, label

# Voxelize

In [4]:
def voxelize3D(pts, dim=[1,1,1]):
    """
    pts: receives .pts cloud point data. 2D array, arbitary sized X,Y,Z pairs. (We will only take x,y,z into account for now)
    dim: dimensioin of output voxelized data
    
    This function will locate the grid cube and calculate the density of each cube.
    The output will be normalized values.
    """
    assert(pts.shape[1]>=3), "pts file should contain at least x,y,z coordinate"
    assert(len(dim)==3), "Please provide 3-d grid size like [32,32,32]"
    
    # move all the axis to positive area.
    minimum_val = [pts[0][0], pts[0][1], pts[0][2]]

    # find the smallest 
    for pair in pts:
        if pair[0] < minimum_val[0]:
            minimum_val[0] = pair[0]
        if pair[1] < minimum_val[1]:
            minimum_val[1] = pair[1]
        if pair[2] < minimum_val[2]:
            minimum_val[2] = pair[2]
            
    # move it to first quadrant 
    rectified_pts = np.empty(pts.shape)
    for index, pair in enumerate(pts):
        point = np.zeros(3)
        point[0] = pair[0] - minimum_val[0]
        point[1] = pair[1] - minimum_val[1]
        point[2] = pair[2] - minimum_val[2]
        rectified_pts[index] = point
    
    # biggest value in each axis 
    maximum_val = pts[0][0]
    
    for pair in rectified_pts:
        for val in pair:
            if val > maximum_val:
                maximum_val = val
     
    # normalize all the axises to (0,1)
    normalized_pts = rectified_pts/maximum_val
    
    x_grid_length = 1/dim[0]
    y_grid_length = 1/dim[1]
    z_grid_length = 1/dim[2]
    
    output = np.zeros((dim[0],dim[1],dim[2]))
    
    epsilon = 0.000000000001 # we will have at least a 1.0 value which will exceed the index of grid
    # we can use a relativly small value to escape that to fit our data
    
    max_volume_size = 0
    
    for pair in normalized_pts:
        x_loc = int(pair[0]/(x_grid_length + epsilon))
        y_loc = int(pair[1]/(y_grid_length + epsilon))
        z_loc = int(pair[2]/(z_grid_length + epsilon))
        if output[x_loc, y_loc, z_loc] is None:
            output[x_loc, y_loc, z_loc] = 1
        else:
            output[x_loc, y_loc, z_loc] += 1
        
        if output[x_loc, y_loc, z_loc] > max_volume_size:
            max_volume_size = output[x_loc, y_loc, z_loc]
    
    output = output/max_volume_size    
            
    return output

In [5]:
data, label = find_data(data_path, folder_filter=(10, 100))

In [6]:
print(len(data), len(label))

297 297


In [14]:
my_point_cloud = PyntCloud.from_file(data[30], sep=" ", header=0, names=["x","y","z"])

In [9]:
def write_to_file(xyz, filename='xyz'):
    import os
    file = open(os.path.join(os.getcwd(), filename + ".pts"), "w") 
    
    for point in xyz:
        st = ""
        for item in point:
            st += str(item) + " "
        file.write(st.strip() + "\n")

    file.close() 

# To hdf5

In [81]:
def get_data(data_path, max_file_num=None, dim=[32,32,32]):
    """
    Output original xyz based points and voxelized data.
    """
    data = []
    
    target_dir_path = os.path.join(data_path, 'points')
    path, dirs, files = os.walk(target_dir_path).__next__()
    file_count = len(files)
    
    count = 0
    for pts_data in os.scandir(target_dir_path):
        if (max_file_num is None) or (count < max_file_num):
            _path = os.path.join(data_path, 'points', pts_data.name)
            pts = PyntCloud.from_file(_path, sep=" ", header=0, names=["x","y","z"])
            
            _vox = voxelize3D(pts.xyz, dim=dim)
            vox_chan = np.array(_vox).reshape(_vox.shape + (1,))
            data.append(vox_chan)
            count += 1
        else:
            break

                
    return np.asarray(data, dtype=np.float32)

In [101]:
import h5py
import os
import numpy as np

def write_big_files_to_h5(data_path, folder_filter=(None, None), dim=[32,32,32]):
    
    data, label = find_data(data_path, folder_filter=folder_filter)
    
    _shuffled_data_raw, _shuffled_label_raw = data_shuffling(data, label)
    
    _shuffled_data = data_reshape(_shuffled_data_raw)
    _shuffled_label, _label_ref = data_onehot_encode(_shuffled_label_raw)

    # Create hdf5
    hdf5_path = os.path.join(data_path, 'small_shuffled_sets.h5')
    hdf5_file = h5py.File(hdf5_path, mode='w')

    hdf5_file.create_dataset("voxels", data=_shuffled_data[:])
    hdf5_file.create_dataset("labels", data=_shuffled_label[:])
    hdf5_file.create_dataset('label_ref', data=np.array(_label_ref).astype('|S9')) # ASCII
            
            
write_big_files_to_h5(data_path, folder_filter=(None, 2000))

Process:   100% 


In [97]:
h5 = h5py.File(os.path.join(data_path, 'big_shuffled_sets.h5'), mode='r')
d = h5.get("voxels")
ll = h5.get('labels')
l = h5.get("label_ref")
print(d.shape, ll[:].shape)

(297, 32, 32, 32, 1) (297, 4)


In [5]:
# Output with binary cells instead of density
import h5py
import os
import numpy as np

def density_converter(data_path, file_name):
    h5 = h5py.File(data_path, mode='r')
    vox = h5.get("voxels")
    _data = []
    for index, value in enumerate(vox):
        idx = value > 0
        val = np.ones(value.shape) * idx
        _data.append(val)
        print("Process:" + str('{0:.2f}'.format(index/len(vox))), end='\r')
    
    _data = np.asarray(_data, dtype=np.float32)
    hdf5_path = os.path.join(os.getcwd(), file_name)
    hdf5_file = h5py.File(hdf5_path, mode='w')

    hdf5_file.create_dataset("voxels", data=_data[:])
    hdf5_file.create_dataset("labels", data=h5.get("labels"))
    hdf5_file.create_dataset('label_ref', data=h5.get("label_ref")) # ASCII
        
# density_converter(os.path.join(os.getcwd(), 'h5dataset', 'big_shuffled_sets.h5'), "test.h5")

In [ ]:
path = os.path.join(os.getcwd(), 'test.h5')
h5 = h5py.File(path, mode='r')
_d = h5.get("voxels")
plot_voxelgrid(_d[100])

In [ ]:
def write_each_folder_to_h5(data_path, dim=[32,32,32]):
    for entry in os.scandir(data_path):
        if entry.is_dir():
            target_dir_path = os.path.join(data_path, entry.name)
            data = get_data(target_dir_path, dim=dim)
            
            labels=np.empty(len(data), dtype='<U9'); # UTF-8
            labels.fill(entry.name)

            # Create hdf5
            hdf5_path = os.path.join(data_path, entry.name + '.h5')
            hdf5_file = h5py.File(hdf5_path, mode='w')

            hdf5_file.create_dataset("voxels", data=data[:])
            hdf5_file.create_dataset('label_ref', data=labels.astype('|S9')) # ASCII
            

In [ ]:
h5 = h5py.File(os.path.join(os.getcwd(), 'h5dataset', 'table.h5'), mode='r')
d = h5.get("points")
l = h5.get("label_ref")
print(d.shape, l[:])

# Ploting

In [7]:
# %load ./voxel_grid_plot.py
import os
import shutil

import numpy as np

from IPython.display import IFrame
from matplotlib import pyplot as plt


def plot_voxelgrid(voxelgrid,
                   output_name=None,
                   cmap="Oranges",
                   axis=True,
                   width=800,
                   height=600):

    scaled_shape = voxelgrid.shape

    vector = voxelgrid
    points = np.argwhere(vector) * scaled_shape

    s_m = plt.cm.ScalarMappable(cmap=cmap)
    rgb = s_m.to_rgba(vector.reshape(-1)[vector.reshape(-1) > 0])

    camera_position = points.max(0) + abs(points.max(0))

    look = points.mean(0)

    if axis:
        axis_size = points.ptp() * 1.5
    else:
        axis_size = 0

    placeholders = {}

    placeholders["POINTS_X_PLACEHOLDER"] = points[:, 0].tolist()
    placeholders["POINTS_Y_PLACEHOLDER"] = points[:, 1].tolist()
    placeholders["POINTS_Z_PLACEHOLDER"] = points[:, 2].tolist()

    placeholders["R_PLACEHOLDER"] = rgb[:, 0].tolist()
    placeholders["G_PLACEHOLDER"] = rgb[:, 1].tolist()
    placeholders["B_PLACEHOLDER"] = rgb[:, 2].tolist()

    placeholders["S_x_PLACEHOLDER"] = scaled_shape[0]
    placeholders["S_y_PLACEHOLDER"] = scaled_shape[1]
    placeholders["S_z_PLACEHOLDER"] = scaled_shape[2]

    placeholders["CAMERA_X_PLACEHOLDER"] = camera_position[0]
    placeholders["CAMERA_Y_PLACEHOLDER"] = camera_position[1]
    placeholders["CAMERA_Z_PLACEHOLDER"] = camera_position[2]

    placeholders["LOOK_X_PLACEHOLDER"] = look[0]
    placeholders["LOOK_Y_PLACEHOLDER"] = look[1]
    placeholders["LOOK_Z_PLACEHOLDER"] = look[2]

    placeholders["AXIS_SIZE_PLACEHOLDER"] = axis_size

    placeholders["N_VOXELS_PLACEHOLDER"] = sum(vector.reshape(-1) > 0)

    if output_name is None:
        output_name = "plotVG.html"

    BASE_PATH = os.getcwd()
    src = "{}/{}".format(BASE_PATH, "voxelgrid.html")
    dst = "{}/{}".format(os.getcwd(), output_name)

    with open(src, "r") as inp, open(dst, "w") as out:
        for line in inp:
            for key, val in placeholders.items():
                if key in line:
                    line = line.replace(key, str(val))
            out.write(line)

    return IFrame(output_name, width=width, height=height)

# Rotation

In [10]:
def rotate(pts, axis=0, angle_degree=30):
    """
    Rotate the pts point cloud by a specific angle.
    
    axis = 0 : rotate around x-axis
    axis = 1 : rotate around y-axis
    axis = 2 : rotate around z-axis
    
    angle_degree : how much degree you want to rotate the point cloud.
    
    Algorithm comes in: https://uk.mathworks.com/matlabcentral/answers/123763-how-to-rotate-entire-3d-data-with-x-y-z-values-along-a-particular-axis-say-x-axis
    """
    if axis == 0: 
        return rotate_by_x(pts, angle_degree)
    if axis == 1: 
        return rotate_by_y(pts, angle_degree)
    if axis == 2: 
        return rotate_by_z(pts, angle_degree)
    
def rotate_by_x(pts, angle_degree=30):
    import numpy as np
    degree = np.deg2rad(angle_degree)
    res = np.empty(pts.shape)
    for index, point in enumerate(pts):
        new_point = np.zeros(3)
        new_point[0] = point[0] # x
        new_point[1] = point[1]*np.cos(degree) - point[2]*np.sin(degree) # y
        new_point[2] = point[1]*np.sin(degree) + point[2]*np.cos(degree) # z
        res[index] = new_point
        
    return res

    
def rotate_by_y(pts, angle_degree=30):
    import numpy as np
    degree = np.deg2rad(angle_degree)
    res = np.empty(pts.shape)
    for index, point in enumerate(pts):
        new_point = np.zeros(3)
        new_point[0] = point[0]*np.cos(degree) + point[2]*np.sin(degree) # x
        new_point[1] = point[1] # y
        new_point[2] = point[2]*np.cos(degree) - point[0]*np.sin(degree) # z
        res[index] = new_point
        
    return res

    
def rotate_by_z(pts, angle_degree=30):
    import numpy as np
    degree = np.deg2rad(angle_degree)
    res = np.empty(pts.shape)
    for index, point in enumerate(pts):
        new_point = np.zeros(3)
        new_point[0] = point[0]*np.cos(degree) - point[1]*np.sin(degree) # x
        new_point[1] = point[0]*np.sin(degree) + point[1]*np.cos(degree) # y
        new_point[2] = point[2] # z
        res[index] = new_point
        
    return res

In [13]:
rotated_point_cloud = rotate(my_point_cloud.xyz, angle_degree=180)
write_to_file(rotated_point_cloud)

In [14]:
rotated_point_cloud_file = PyntCloud.from_file(os.path.join(os.getcwd(), "xyz.pts"), sep=" ", header=0, names=["x","y","z"])
rotated_point_cloud_file.plot(point_size=0.005)

# Squeeze

In [15]:
def squeeze_pts(pts, axis=0, percentage=10):
    if axis==0:
        return squeeze_by_x(pts, percentage)
    if axis==1:
        return squeeze_by_y(pts, percentage)
    if axis==2:
        return squeeze_by_z(pts, percentage)
    
def squeeze_by_x(pts, percentage):
    
    v = (100-percentage)/100
    output = np.empty(pts.shape)
    
    assert(v>=0 and v<=1), "Percentage should between 0 and 100"
    
    for index, point in enumerate(pts):
        output[index] = point
        output[index][0] = output[index][0]*v
        
    return output
    
def squeeze_by_y(pts, percentage):
    
    v = (100-percentage)/100
    output = np.empty(pts.shape)
    
    assert(v>=0 and v<=1), "Percentage should between 0 and 100"
    
    for index, point in enumerate(pts):
        output[index] = point
        output[index][1] = output[index][1]*v
        
    return output

    
def squeeze_by_z(pts, percentage):
    
    v = (100-percentage)/100
    output = np.empty(pts.shape)
    
    assert(v>=0 and v<=1), "Percentage should between 0 and 100"
    
    for index, point in enumerate(pts):
        output[index] = point
        output[index][2] = output[index][2]*v
        
    return output

In [16]:
squeezed_pts = squeeze_pts(my_point_cloud.xyz, axis=1, percentage=30)
write_to_file(squeezed_pts, filename="sq")

In [17]:
rotated_point_cloud_file = PyntCloud.from_file(os.path.join(os.getcwd(), "sq.pts"), sep=" ", header=0, names=["x","y","z"])
rotated_point_cloud_file.plot(point_size=0.005)

# Gaussian Noise

In [22]:
def add_noise(pts, noise_percentage=0.05):
    
    noise_size = int(noise_percentage*len(pts))
    
    max_x = min_x = pts[0][0]
    max_y = min_y = pts[0][1]
    max_z = min_z = pts[0][2]
    
    for point in pts:
        if max_x < point[0]:
            max_x = point[0]
        if max_y < point[1]:
            max_y = point[1]
        if max_z < point[2]:
            max_z = point[2]
            
        if min_x < point[0]:
            min_x = point[0]
        if min_y < point[1]:
            min_y = point[1]
        if min_y < point[2]:
            min_y = point[2]
            
    import numpy as np
    
    noise_pts = np.empty((noise_size, 3))
    noise_x = np.random.randn(noise_size)*(max_x - min_x)/2
    noise_y = np.random.randn(noise_size)*(max_y - min_y)/2
    noise_z = np.random.randn(noise_size)*(max_z - min_z)/2
    
    for i, _ in enumerate(range(noise_size)):
        pos = np.random.randint(0,len(pts)-1)
        noise_pts[i][0] = pts[pos][0] + noise_x[i]
        noise_pts[i][1] = pts[pos][1] + noise_y[i]
        noise_pts[i][2] = pts[pos][2] + noise_z[i]
        
    return np.concatenate((pts, noise_pts))

In [23]:
noisy_pts = add_noise(my_point_cloud.xyz)
points_df = pd.DataFrame(noisy_pts, columns=["x", "y", "z"])
PyntCloud(points_df).plot(point_size=0.005)

# Data Reading & preprocessing

In [89]:
def label_cates(labels):
    cates = []
    for l in labels:
        if l not in cates:
            cates.append(l)
    return cates

def data_onehot_encode(labels):
    """
    Recieves an array of labels.
    """
    cates = label_cates(labels)
    # one-hot
    onehot = []
    for l in labels:
        x = np.zeros(len(cates))
        x[cates.index(l)] = 1.0
        onehot.append(x)
    return onehot, cates

In [90]:
def data_reshape(data, dim=[32,32,32]):
    """
    Will read and voxelize the data
    """
    x_reshaped = []
    
    for i in range(len(data)):
        if i % 20 == 0:
            print("Process: ", str('{0:.2f}'.format(i/len(data))) + "%", end='\r')
        my_point_cloud = PyntCloud.from_file(data[i], sep=" ", header=0, names=["x","y","z"])
        vox = voxelize3D(my_point_cloud.xyz, dim=dim)
        vox_chan = np.array(vox).reshape(vox.shape + (1,))
        x_reshaped.append(vox_chan)
        
    print("Process: ", " 100% ")        
    return x_reshaped

# Data Shuffling

In [91]:
def data_swap(data, index_a, index_b):
    temp = data[index_a]
    data[index_a] = data[index_b]
    data[index_b] = temp
    return data

def data_random_position(data):
    import random
    return random.randint(0,len(data)-1)

def data_shuffling(data, label):
    for i in range(len(data)):
        target = data_random_position(data)
        data = data_swap(data, i, target)
        label = data_swap(label, i, target)
    return data, label

In [24]:
shuffled_data_raw, shuffled_label_raw = data_shuffling(data, label)

In [49]:
shuffled_data = data_reshape(shuffled_data_raw)
shuffled_label, label_ref = data_onehot_encode(shuffled_label_raw)

Process:   100% 


In [50]:
print(shuffled_data[100].shape)
print(shuffled_label[100])
print(len(shuffled_label[100]))
print(label_ref)

(32, 32, 32, 1)
[ 0.  1.  0.  0.]
4
['rocket', 'cap', 'bag', 'earphone']


In [60]:
import h5py
import os
import tensorflow as tf

# Create hdf5
hdf5_path = os.path.join(os.getcwd(), 'small_dataset.h5')
hdf5_file = h5py.File(hdf5_path, mode='w')

hdf5_file.create_dataset("points", data=shuffled_data[:])
hdf5_file.create_dataset("labels", data=shuffled_label[:])
hdf5_file.create_dataset('label_ref', data=np.array(label_ref).astype('|S9'))
# hdf5_file.create_dataset("label_ref", data=label_ref[:], dtype=h5py.special_dtype(vlen=str))

<HDF5 dataset "label_ref": shape (4,), type "|S9">

In [62]:
h5 = h5py.File(hdf5_path, mode='r')
d = h5.get("points")
l = h5.get("labels")
re = h5.get("label_ref")
print(d[30][:,:,:,-1].shape)
print(re[:])
plot_voxelgrid(d[30][:,:,:,-1])

(32, 32, 32)
[b'rocket' b'cap' b'bag' b'earphone']


# Cropping data

In [ ]:
def norm_point(point_cloud):
    
    ((x_min, x_max), (y_min, y_max), (z_min, z_max)) = find_ranges(point_cloud)
    
    biggest_value = np.max(np.asarray([x_max - x_min, y_max - y_min, z_max - z_min])) + 0.000000001
    
    normalized_points = []
    for ind, coor in enumerate(point_cloud):
        row = np.empty(len(coor))
        row[0] = (coor[0] - x_min)/biggest_value
        row[1] = (coor[1] - y_min)/biggest_value
        row[2] = (coor[2] - z_min)/biggest_value
        normalized_points.append(row)
        
    return np.asarray(normalized_points)

def crop(point_cloud, ratio=((0, 1), (0, 1), (0, 1))):
    
    normed_point = norm_point(point_cloud)
    
    res = []
    
    for idx, row in enumerate(normed_point):
        if row[0] > ratio[0][0] and row[0] < ratio[0][1]:
            if row[1] > ratio[1][0] and row[1] < ratio[1][1]:
                if row[2] > ratio[2][0] and row[2] < ratio[2][1]:
                    res.append(row)
                    
    return np.asarray(res)